# **Two fold cross validation for logistic regression and baseline**

## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_score
import pandas as pd

## Load dataset

In [2]:
filename = 'heart_failure_clinical_records_dataset.csv'
df = pd.read_csv(filename)
y = df['DEATH_EVENT']
X = df.drop(['DEATH_EVENT'], axis = 1)
attributeNames = list(X.keys())

classNames = "DEATH_EVENT"
N, M = X.shape
C = len(classNames)

## Define our models

We are going to define two models:

- A baseline
- A logistic regression 

### Baseline

In [ ]:
# In our dataset there are more entries of the class 0 (Survived)
# The best possible accuracy will be around 70% given our dataset
def baseline():
    return 0


### Logistic regression

In [ ]:
regularization_force = 1/1e-8
log_reg = lambda regularization: LogisticRegression(penalty="l2", C= 1/regularization)

## Two fold cross-validation

In [ ]:
K1 = 10 # Number of outer folds
K2 = 10 # Number of inner folds
error_LG = []

for i in range(K1):
    print('\nCrossvalidation outer fold: {0}/{1}'.format(i+1,K1))    
    X_train_f1, X_test_f1, y_train_f1, y_test_f1 = train_test_split(X, y, test_size=0.8, stratify=y)

    # Standardize the training set based on training set mean and std
    mu = np.mean(X_train, 0)
    sigma = np.std(X_train, 0)

    X_train = (X_train - mu) / sigma
    X_test = (X_test - mu) / sigma

    innerKfold = StratifiedKFold(n_splits= K2, shuffle=True)
    for k, train_index, test_index in enumerate(innerKfold.split(X_train_f1, y_train_f1)):
        X_train_f2 = X_train_f1[train_index]
        X_test_f2 = X_test_f1[train_index]
        y_train_f2 = y_train_f1[train_index]
        y_test_f2 = y_test_f1[test_index]

        log_reg()

    
